Install requirements

In [ ]:
!pip install requests


In [ ]:
import pandas as pd
import json

In [ ]:
import requests

import json file

In [ ]:
URL = "https://raw.githubusercontent.com/francoisbonnard/ayomi/master/DATASET%20EMAILS%20-%20Raw.json"

In [ ]:
response = requests.get(URL)

In [ ]:
if response.status_code == 200:
    data = json.loads(response.text)
else:
    print("Failed to retrieve data:", response.status_code)


In [ ]:
json_string = json.dumps(data)

In [ ]:
df = pd.read_json(json_string)


Check data

In [ ]:
df.head()

,body,from
0,"Chers clients, nous avons le plaisir de vous i...",blaine.lang@gmail.com
1,"Salut Marie, ça fait un bail ! Comment vas-tu ...",edgar.dickinson@gmail.com
2,"Bonjour, nous avons trouvé des erreurs dans vo...",loran41@yahoo.com
3,"Hey Julien, ça fait longtemps qu'on ne s'est p...",marshal37@blickcarter.com
4,"Bonjour, je suis actuellement en train de prép...",anika.trantow@kris.org


In [ ]:
occurrences = df['from'].value_counts()
print(occurrences)

from
kmorar@yahoo.com                    4
aorn@yahoo.com                      4
epouros@yahoo.com                   4
sbrown@hotmail.com                  4
kschumm@gmail.com                   4
                                   ..
brekke.jami@altenwerthgraham.com    1
aidyn.kuphal@gmail.com              1
elease.skiles@rempel.com            1
zgaylord@heaney.com                 1
aschulist@gmail.com                 1
Name: count, Length: 98108, dtype: int64


In [ ]:
filtered_df = df[df['from'] == 'kmorar@yahoo.com']
print(filtered_df['body'])



2739     Salut Jeffrey,\nJ'espère que tu vas bien! Peux...
10861    Chers clients, nous avons le plaisir de vous i...
42442    Chers clients, nous sommes ravis de vous annon...
90016    Bonjour, nous avons bien reçu votre commande. ...
Name: body, dtype: object


Vérifier les séquences d'échappement Unicode

In [ ]:
df.loc[1]

body                  Salut Marie, ça fait un bail ! Comment vas-tu ...
from                                          edgar.dickinson@gmail.com
sentiment analysis                                             NEGATIVE
Name: 1, dtype: object

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)


In [ ]:
df.loc[1]

body                  Salut Marie, ça fait un bail ! Comment vas-tu ? J'ai eu ton numéro par Jacques et je me suis dit qu'on pourrait se revoir autour d'un verre pour discuter des vieux temps. Fais-moi signe si tu es partante ! À bientôt, Jean.
from                                                                                                                                                                                                                       edgar.dickinson@gmail.com
sentiment analysis                                                                                                                                                                                                                          NEGATIVE
Name: 1, dtype: object

Import transformers

In [ ]:
! pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


Import pipeline et sentiment-analysis en francais


In [ ]:
from transformers import pipeline


In [ ]:
analyzer = pipeline(
    task='text-classification',
    model="cmarkea/distilcamembert-base-sentiment",
    tokenizer="cmarkea/distilcamembert-base-sentiment"
)

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/272M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

ajoute une colonne au dataframe


In [ ]:
df['sentiment analysis'] = "none"


In [ ]:
df.head()

,body,from,sentiment analysis
0,"Chers clients, nous avons le plaisir de vous informer que votre commande a été expédiée. Vous pouvez suivre votre colis en cliquant sur le lien suivant : [lien de suivi]. Merci de votre confiance.",blaine.lang@gmail.com,none
1,"Salut Marie, ça fait un bail ! Comment vas-tu ? J'ai eu ton numéro par Jacques et je me suis dit qu'on pourrait se revoir autour d'un verre pour discuter des vieux temps. Fais-moi signe si tu es partante ! À bientôt, Jean.",edgar.dickinson@gmail.com,none
2,"Bonjour, nous avons trouvé des erreurs dans votre déclaration d'impôts. Cliquez ici pour corriger ces erreurs et éviter des pénalités.",loran41@yahoo.com,none
3,"Hey Julien, ça fait longtemps qu'on ne s'est pas vus ! On devrait organiser une soirée jeux de société chez moi ce week-end. Tu serais partant ? Dis-moi si tu peux venir. À bientôt, Olivier.",marshal37@blickcarter.com,none
4,"Bonjour, je suis actuellement en train de préparer une réunion avec notre équipe et j'aurais besoin de votre aide pour établir l'ordre du jour. Pourriez-vous me donner vos suggestions ? Merci d'avance, Olivier.",anika.trantow@kris.org,none


In [ ]:
def apply_sentiment_analysis(text):
    return analyzer(text)[0]['label']


teste le classifier sur les 10 premières lignes

In [ ]:
%time df.loc[:9, 'sentiment analysis'] = df.loc[:9, 'body'].apply(apply_sentiment_analysis)


CPU times: user 1.54 s, sys: 5.16 ms, total: 1.55 s
Wall time: 2.54 s


In [ ]:
df

,body,from,sentiment analysis
0,"Chers clients, nous avons le plaisir de vous informer que votre commande a été expédiée. Vous pouvez suivre votre colis en cliquant sur le lien suivant : [lien de suivi]. Merci de votre confiance.",blaine.lang@gmail.com,5 stars
1,"Salut Marie, ça fait un bail ! Comment vas-tu ? J'ai eu ton numéro par Jacques et je me suis dit qu'on pourrait se revoir autour d'un verre pour discuter des vieux temps. Fais-moi signe si tu es partante ! À bientôt, Jean.",edgar.dickinson@gmail.com,4 stars
2,"Bonjour, nous avons trouvé des erreurs dans votre déclaration d'impôts. Cliquez ici pour corriger ces erreurs et éviter des pénalités.",loran41@yahoo.com,3 stars
3,"Hey Julien, ça fait longtemps qu'on ne s'est pas vus ! On devrait organiser une soirée jeux de société chez moi ce week-end. Tu serais partant ? Dis-moi si tu peux venir. À bientôt, Olivier.",marshal37@blickcarter.com,4 stars
4,"Bonjour, je suis actuellement en train de préparer une réunion avec notre équipe et j'aurais besoin de votre aide pour établir l'ordre du jour. Pourriez-vous me donner vos suggestions ? Merci d'avance, Olivier.",anika.trantow@kris.org,4 stars
...,...,...,...
99995,"Bonjour Papa, j'ai besoin de ton avis sur quelque chose. Je me demande si je devrais accepter cette nouvelle offre d'emploi. Pourrais-tu m'appeler ce soir pour en discuter ? Merci, Isabelle.",classie.friesen@gmail.com,none
99996,"Madame, Monsieur, je vous écris car je souhaite résilier mon abonnement à votre service. Pouvez-vous m'indiquer la marche à suivre ? Cordialement, François.",tromp.terell@gmail.com,none
99997,"Salut Marie, ça fait un bail ! Comment vas-tu ? J'ai eu ton numéro par Jacques et je me suis dit qu'on pourrait se revoir autour d'un verre pour discuter des vieux temps. Fais-moi signe si tu es partante ! À bientôt, Jean.",zaire79@lueilwitzstehr.com,none
99998,"Madame, Monsieur, je vous écris car je souhaite vous faire part de mon intérêt pour le projet que vous dirigez. Pouvez-vous me donner plus d'informations sur les modalités de participation ? Cordialement, David.",lorean.jakubowski@kshlerin.com,none


In [ ]:
df.shape

(100000, 3)

temps de process de 1000 lignes

In [ ]:
%time df.loc[9:1000, 'sentiment analysis'] = df.loc[9:1000, 'body'].apply(apply_sentiment_analysis)

CPU times: user 1min 49s, sys: 216 ms, total: 1min 49s
Wall time: 1min 53s


In [ ]:
df.loc[90:100]

,body,from,sentiment analysis
90,"Madame, Monsieur, je vous écris car je souhaite vous faire part d'un problème que j'ai rencontré dans le cadre de mon travail. Pouvons-nous en discuter pour trouver une solution ? Cordialement, Sophie.",cronin.nickolas@yahoo.com,3 stars
91,"Bonjour, je suis actuellement en train de préparer une présentation pour le comité de direction et j'aurais besoin de votre aide pour rassembler certaines données. Pourriez-vous me les fournir d'ici la fin de la semaine ? Merci d'avance, Benjamin.",gutmann.mario@gmail.com,3 stars
92,"Bonjour, nous avons trouvé des virus sur votre ordinateur. Cliquez ici pour nettoyer votre ordinateur et protéger vos données.",ammie28@rolfson.com,1 star
93,"Madame, Monsieur, je vous contacte car je suis intéressé par votre entreprise et je souhaiterais en savoir plus sur vos produits/services. Pouvez-vous me donner un rendez-vous pour en discuter ? Cordialement, Élisabeth.",daryl.leffler@yahoo.com,4 stars
94,"Hey Nicolas, j'espère que tu vas bien. Je t'écris car j'ai besoin de tes conseils pour choisir un nouvel appareil photo. Tu es tellement passionné par la photographie, je suis sûr que tu pourras m'aider. On pourrait se voir ce week-end ? À bientôt, Marie.",abbott.lauri@cole.com,5 stars
95,"Coucou Julia,\nJe t'ai envoyé les dernières photos de Marco, il grandit si vite! Regarde comme il sourit sur celle avec le chien.\nBisous,\nCarlos",acruickshank@powlowskikunde.org,5 stars
96,"Chers abonnés, nous sommes ravis de vous annoncer le lancement de notre nouveau site web. Découvrez nos nouveaux produits et services, ainsi que nos offres spéciales pour les premiers visiteurs. Cliquez ici pour découvrir notre nouveau site.",zaiden.ortiz@trompyundt.biz,5 stars
97,"Coucou Sophie, ça fait longtemps ! Comment vas-tu ? J'ai vu que tu étais allée à un concert génial la semaine dernière. Raconte-moi tout ! Bisous, Laura.",conn.branson@rempelnikolaus.com,5 stars
98,"Coucou Julia,\nJe t'ai envoyé les dernières photos de Marco, il grandit si vite! Regarde comme il sourit sur celle avec le chien.\nBisous,\nCarlos",rath.johnnie@gmail.com,5 stars
99,"Hey Guillaume, j'espère que tu vas bien. Je t'écris car j'ai besoin de tes conseils pour choisir une nouvelle voiture. Tu es tellement calé en mécanique, je suis sûr que tu pourras m'aider. On pourrait se voir ce week-end ? Bisous, Aurélie.",katherin71@gmail.com,5 stars
